# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

## LLMChain

In [2]:
import os
import openai
import json

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.getenv("OPENAI_API_KEY") 

# check the sever type according to API key
if openai.api_key.startswith("sk-"): # OpenAI
    server_type = "OpenAI"
else:
    server_type = "AzureOpenAI"

# update env info if serer type is AzureOpenAI
if server_type == "AzureOpenAI": # AzureOpenAI
    # Load config values
    with open(r'../azure_config.json') as config_file:
        config_details = json.load(config_file)
        # Set AzureOpenAI API values
        openai.api_type = config_details["OPENAI_API_TYPE"]
        os.environ['OPENAI_API_BASE'] = config_details["OPENAI_API_BASE"]
        openai.api_base=config_details["OPENAI_API_BASE"]
        os.environ["OPENAI_API_VERSION"] = config_details["OPENAI_API_VERSION"]
        openai.api_version=config_details["OPENAI_API_VERSION"]

# check the enironment variable
print(f"sever_type: {server_type} | api_type: {openai.api_type}")

sever_type: AzureOpenAI | api_type: azure


In [3]:
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [4]:
# construct chat function according to API key format
if server_type == "OpenAI":
    llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k",temperature=0.9)
else:
    llm=AzureChatOpenAI(deployment_name="gpt-35-turbo",model_name="gpt-35-turbo", temperature=0.9)

In [5]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)
# print(prompt)
chain = LLMChain(llm=llm, prompt=prompt)

In [6]:
product = "Queen Size Sheet Set"
print(chain.run(product))

Royal Comfort Sheets


## SimpleSequentialChain

In [7]:
from langchain.chains import SimpleSequentialChain

In [8]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [9]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [10]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [11]:
overall_simple_chain.run(product)

Error in on_chain_start callback: 'name'


Regal Bedding Co.
Regal Bedding Co. is a leading manufacturer of luxurious bedding products that offer exceptional comfort and quality.

> Finished chain.


'Regal Bedding Co. is a leading manufacturer of luxurious bedding products that offer exceptional comfort and quality.'

## SequentialChain

In [12]:
from langchain.chains import SequentialChain

In [13]:
# construct chat function according to API key format
if server_type == "OpenAI":
    llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k",temperature=0.9)
else:
    llm=AzureChatOpenAI(deployment_name="gpt-35-turbo", model_name="gpt-35-turbo", temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [14]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

In [15]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [16]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [17]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [18]:
import pandas as pd
df = pd.read_csv('Data.csv')
display(df)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


In [19]:
review = df.Review[0]
review

'I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.'

In [20]:
overall_chain(review)

Error in on_chain_start callback: 'name'



> Finished chain.


{'Review': 'I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.',
 'English_Review': "I ordered a king size set, but my only complaint is that I wish the seller would offer it with four pillowcases instead of two. I ended up ordering a separate two pack of pillowcases to have a total of four, but when they arrived, the color looked slightly different. However, customer service was great and sent me two more pillowcases that matched perfectly. I'm happy with the quality and cooling properties of the sheets, especially for the price.",
 'summary': 'The re

## Router Chain

In [21]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [22]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [23]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [24]:
if server_type == "OpenAI":
    llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k",temperature=0.9)
else:
    llm=AzureChatOpenAI(deployment_name="gpt-35-turbo", model_name="gpt-35-turbo", temperature=0.9)

In [25]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [26]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [27]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [28]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [29]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [30]:
chain.run("What is black body radiation?")

Error in on_chain_start callback: 'name'


physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized object that absorbs all radiation that falls on it and emits radiation at all frequencies and wavelengths. The amount and distribution of the radiation emitted by a black body depend on its temperature, with higher temperatures producing more radiation at shorter wavelengths. Black body radiation is important in many areas of physics, including the study of stars, the behavior of materials at high temperatures, and the development of quantum mechanics.'

In [31]:
chain.run("what is 2 + 2")

Error in on_chain_start callback: 'name'


math: {'input': 'what is 2 + 2'}
> Finished chain.


'As an AI language model, I can confidently say that the answer to "what is 2 + 2" is 4.'

In [32]:
chain.run("Why does every cell in our body contain DNA?")

Error in on_chain_start callback: 'name'


None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA contains the genetic information that instructs cells how to function and replicate. DNA carries all the information necessary to produce and maintain an entire organism, including its physical and biochemical characteristics. The genetic information in DNA is passed on from parent to offspring during reproduction, ensuring that the genetic information is preserved and passed on to future generations. Therefore, every cell in our body needs to contain DNA to ensure that the genetic information is available and accessible to carry out the necessary cellular processes and functions.'